# 🎬 GNN-Based Movie Recommendation System
## Using LightGCN on MovieLens 100K Dataset

This notebook builds a **Graph Neural Network** recommender where:
- **Nodes**: Users and Movies  
- **Edges**: User-Movie interactions (ratings ≥ 4)

### What You'll Learn:
1. How to construct a bipartite graph for recommendations
2. How LightGCN learns embeddings through message passing
3. How peer influence propagates through multi-hop connections
4. How to generate and evaluate recommendations

## Part 1: Setup and Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from collections import defaultdict
import urllib.request
import zipfile
import os
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## Part 2: Download and Load MovieLens 100K

The dataset contains:
- **100,000 ratings** from 943 users on 1,682 movies
- User demographics (age, gender, occupation)
- Movie genres (19 genres as one-hot vectors)

In [2]:
def download_movielens():
    data_dir = 'ml-100k'
    if not os.path.exists(data_dir):
        print("Downloading MovieLens 100K...")
        url = 'https://files.grouplens.org/datasets/movielens/ml-100k.zip'
        urllib.request.urlretrieve(url, 'ml-100k.zip')
        with zipfile.ZipFile('ml-100k.zip', 'r') as z:
            z.extractall('.')
        os.remove('ml-100k.zip')
        print("Done!")
    return data_dir

def load_data(data_dir):
    ratings = pd.read_csv(f'{data_dir}/u.data', sep='\t',
                         names=['user_id', 'movie_id', 'rating', 'timestamp'])
    users = pd.read_csv(f'{data_dir}/u.user', sep='|',
                       names=['user_id', 'age', 'gender', 'occupation', 'zip'],
                       encoding='latin-1')
    movies = pd.read_csv(f'{data_dir}/u.item', sep='|', encoding='latin-1',
                        names=['movie_id', 'title', 'release', 'video_release', 'url',
                               'unknown', 'Action', 'Adventure', 'Animation', 'Children',
                               'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
                               'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance',
                               'Sci-Fi', 'Thriller', 'War', 'Western'])
    print(f"Ratings: {len(ratings):,}, Users: {len(users)}, Movies: {len(movies)}")
    return ratings, users, movies

data_dir = download_movielens()
ratings_df, users_df, movies_df = load_data(data_dir)

Done!
Ratings: 100,000, Users: 943, Movies: 1682


## Part 3: Build the Bipartite Graph

### What is a Bipartite Graph?
A graph with **two node types** where edges only connect **different types**:

```
Users              Movies
  U1 ───────────── M1 (Toy Story)
  U2 ─────┬─────── M2 (Star Wars)  
  U3 ─────┴─────── M3 (Matrix)
```

Users connect to movies they rated highly (≥4 stars).

### Why This Works for Recommendations:
- After message passing, **similar users** get **similar embeddings**
- A user's embedding naturally "points toward" movies they'd like!

In [ ]:
class MovieLensGraph:
    def __init__(self, ratings_df, users_df, movies_df, threshold=4):
        self.threshold = threshold
        
        # Create ID mappings
        self.user_ids = ratings_df['user_id'].unique()
        self.movie_ids = ratings_df['movie_id'].unique()
        self.user_to_idx = {u: i for i, u in enumerate(self.user_ids)}
        self.movie_to_idx = {m: i for i, m in enumerate(self.movie_ids)}
        self.idx_to_user = {i: u for u, i in self.user_to_idx.items()}
        self.idx_to_movie = {i: m for m, i in self.movie_to_idx.items()}
        
        self.num_users = len(self.user_ids)
        self.num_movies = len(self.movie_ids)
        self.num_nodes = self.num_users + self.num_movies
        
        self.users_df = users_df
        self.movies_df = movies_df
        self.genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy',
                      'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
                      'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                      'Thriller', 'War', 'Western']
        
        # Build edges from positive interactions
        pos = ratings_df[ratings_df['rating'] >= threshold]
        user_idx = [self.user_to_idx[u] for u in pos['user_id']]
        movie_idx = [self.num_users + self.movie_to_idx[m] for m in pos['movie_id']]
        
        # Bidirectional edges
        self.edge_index = torch.tensor([user_idx + movie_idx, movie_idx + user_idx])
        
        # Store user interactions
        self.user_items = defaultdict(set)
        for _, r in pos.iterrows():
            self.user_items[self.user_to_idx[r['user_id']]].add(self.movie_to_idx[r['movie_id']])
        
        print(f"Graph: {self.num_users} users, {self.num_movies} movies, {len(pos)} positive edges")
    
    def train_test_split(self, test_ratio=0.2):
        train_e, test_e = [], []
        for u, movies in self.user_items.items():
            m_list = list(movies)
            np.random.shuffle(m_list)
            n_test = max(1, int(len(m_list) * test_ratio)) if len(m_list) > 1 else 0
            test_e.extend([(u, m + self.num_users) for m in m_list[:n_test]])
            train_e.extend([(u, m + self.num_users) for m in m_list[n_test:]])
        
        src = [e[0] for e in train_e] + [e[1] for e in train_e]
        dst = [e[1] for e in train_e] + [e[0] for e in train_e]
        self.train_edges = torch.tensor([src, dst])
        self.test_edges = test_e
        print(f"Train: {len(train_e)} edges, Test: {len(test_e)} edges")
        return self.train_edges, self.test_edges

graph = MovieLensGraph(ratings_df, users_df, movies_df)
train_edges, test_edges = graph.train_test_split()

Graph: 943 users, 1682 movies, 55375 positive edges
Train: 44675 edges, Test: 10700 edges


## Part 4: LightGCN Model

### How LightGCN Works:

1. **Start**: Each user/movie has a random embedding vector
2. **Message Passing**: Each node aggregates neighbors' embeddings  
3. **Multi-layer**: Stack K layers to capture K-hop neighborhoods
4. **Final Embedding**: Average of all layer outputs

### The Key Formula:
```
e_new = Σ (1/√deg_i × 1/√deg_j) × e_neighbor
```

**No weights, no activations** - just normalized aggregation!

### Why Simpler is Better:
Standard GCNs have weight matrices and activations. For recommendations (where we learn embeddings from scratch), this complexity causes overfitting. LightGCN removes it.

In [ ]:
class LightGCNConv(MessagePassing):
    """LightGCN layer: aggregates neighbor embeddings with degree normalization."""
    def __init__(self):
        super().__init__(aggr='add')
    
    def forward(self, x, edge_index):
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        return self.propagate(edge_index, x=x, norm=norm)
    
    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j


class LightGCN(nn.Module):
    def __init__(self, num_users, num_items, embed_dim=64, num_layers=3):
        super().__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.embedding = nn.Embedding(num_users + num_items, embed_dim)
        nn.init.xavier_uniform_(self.embedding.weight)
        self.convs = nn.ModuleList([LightGCNConv() for _ in range(num_layers)])
    
    def forward(self, edge_index):
        x = self.embedding.weight
        all_emb = [x]
        for conv in self.convs:
            x = conv(x, edge_index)
            all_emb.append(x)
        return torch.stack(all_emb, dim=1).mean(dim=1)  # Average all layers
    
    def get_embeddings(self, edge_index):
        emb = self.forward(edge_index)
        return emb[:self.num_users], emb[self.num_users:]
    
    def predict(self, users, items, edge_index):
        u_emb, i_emb = self.get_embeddings(edge_index)
        return (u_emb[users] * i_emb[items]).sum(dim=1)  # Dot product

model = LightGCN(graph.num_users, graph.num_movies, embed_dim=64, num_layers=3).to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

## Part 5: Training with BPR Loss

### BPR (Bayesian Personalized Ranking):

For each triplet `(user, positive_item, negative_item)`:
```
Loss = -log(sigmoid(score_pos - score_neg))
```

This pushes **positive items to rank higher** than negative items.

### Why BPR instead of MSE?
- We care about **ranking**, not exact rating prediction
- Directly optimizes what we want: correct ordering

In [ ]:
def bpr_loss(pos_scores, neg_scores):
    return -torch.mean(F.logsigmoid(pos_scores - neg_scores))

def sample_negatives(users, num_items, user_items):
    negs = []
    for u in users.cpu().numpy():
        while True:
            n = np.random.randint(num_items)
            if n not in user_items.get(u, set()):
                negs.append(n)
                break
    return torch.tensor(negs)

def train_epoch(model, optimizer, edges, graph, device, batch_size=1024):
    model.train()
    pos = edges[:, edges[0] < graph.num_users]
    perm = torch.randperm(pos.shape[1])
    pos = pos[:, perm]
    
    total_loss = 0
    n_batches = 0
    for i in range(0, pos.shape[1], batch_size):
        batch = pos[:, i:i+batch_size]
        users = batch[0].to(device)
        pos_items = (batch[1] - graph.num_users).to(device)
        neg_items = sample_negatives(users, graph.num_movies, graph.user_items).to(device)
        
        pos_scores = model.predict(users, pos_items, edges.to(device))
        neg_scores = model.predict(users, neg_items, edges.to(device))
        
        loss = bpr_loss(pos_scores, neg_scores)
        reg = 1e-4 * (model.embedding.weight.norm(2) ** 2) / len(users)
        
        optimizer.zero_grad()
        (loss + reg).backward()
        optimizer.step()
        total_loss += loss.item()
        n_batches += 1
    
    return total_loss / n_batches

## Part 6: Evaluation Metrics

- **Hit@K**: Did ANY test item appear in top-K recommendations?
- **NDCG@K**: Normalized ranking quality (rewards items ranked higher)

In [ ]:
def evaluate(model, edges, graph, test_edges, Ks=[5, 10, 20]):
    model.eval()
    with torch.no_grad():
        u_emb, i_emb = model.get_embeddings(edges.to(device))
        u_emb, i_emb = u_emb.cpu(), i_emb.cpu()
    
    user_tests = defaultdict(set)
    for u, m in test_edges:
        user_tests[u].add(m - graph.num_users)
    
    hits = {k: [] for k in Ks}
    ndcgs = {k: [] for k in Ks}
    
    for u, test_items in user_tests.items():
        scores = torch.mm(u_emb[u:u+1], i_emb.t()).squeeze()
        for m in graph.user_items.get(u, []):
            scores[m] = float('-inf')  # Mask seen items
        
        _, topk = torch.topk(scores, max(Ks))
        topk = topk.numpy()
        
        for k in Ks:
            top = set(topk[:k])
            hits[k].append(1.0 if top & test_items else 0.0)
            dcg = sum(1/np.log2(i+2) for i, m in enumerate(topk[:k]) if m in test_items)
            idcg = sum(1/np.log2(i+2) for i in range(min(len(test_items), k)))
            ndcgs[k].append(dcg/idcg if idcg > 0 else 0)
    
    results = {}
    for k in Ks:
        results[f'Hit@{k}'] = np.mean(hits[k])
        results[f'NDCG@{k}'] = np.mean(ndcgs[k])
    return results

## Part 7: Train the Model

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
losses, metrics_history = [], []

print("Training LightGCN...")
print("-" * 50)

for epoch in range(50):
    loss = train_epoch(model, optimizer, train_edges, graph, device)
    losses.append(loss)
    
    if (epoch + 1) % 10 == 0:
        metrics = evaluate(model, train_edges, graph, test_edges)
        metrics_history.append(metrics)
        print(f"Epoch {epoch+1:3d} | Loss: {loss:.4f} | Hit@10: {metrics['Hit@10']:.4f} | NDCG@10: {metrics['NDCG@10']:.4f}")

print("-" * 50)
print("\nFinal Results:")
final = evaluate(model, train_edges, graph, test_edges)
for k, v in final.items():
    print(f"  {k}: {v:.4f}")

## Part 8: Generate Recommendations

Now we can recommend movies to any user by computing similarity between their embedding and all movie embeddings!

In [ ]:
def recommend(model, user_idx, graph, edges, top_k=10):
    model.eval()
    with torch.no_grad():
        u_emb, i_emb = model.get_embeddings(edges.to(device))
    
    scores = torch.mm(u_emb[user_idx:user_idx+1].cpu(), i_emb.cpu().t()).squeeze()
    
    # Mask movies already seen
    for m in graph.user_items.get(user_idx, []):
        scores[m] = float('-inf')
    
    _, top_idx = torch.topk(scores, top_k)
    
    recs = []
    for idx in top_idx.numpy():
        mid = graph.idx_to_movie[idx]
        title = movies_df[movies_df['movie_id'] == mid].iloc[0]['title']
        recs.append((title, scores[idx].item()))
    return recs

# Demo: Recommend for a random user
user = np.random.randint(graph.num_users)
uid = graph.idx_to_user[user]
user_info = users_df[users_df['user_id'] == uid].iloc[0]

print(f"\n👤 User {uid}: {user_info['age']}yo {user_info['gender']}, {user_info['occupation']}")

# Show what they liked
print("\n🎬 Movies they liked:")
for m in list(graph.user_items[user])[:5]:
    mid = graph.idx_to_movie[m]
    title = movies_df[movies_df['movie_id'] == mid].iloc[0]['title']
    print(f"   - {title}")

# Show recommendations  
print("\n✨ Top 10 Recommendations:")
for i, (title, score) in enumerate(recommend(model, user, graph, train_edges), 1):
    print(f"   {i:2d}. {title} (score: {score:.3f})")

## Part 9: Visualize Results

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(losses, 'b-', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('BPR Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

epochs = list(range(10, 51, 10))
hit10 = [m['Hit@10'] for m in metrics_history]
ndcg10 = [m['NDCG@10'] for m in metrics_history]
axes[1].plot(epochs, hit10, 'b-o', label='Hit@10', linewidth=2)
axes[1].plot(epochs, ndcg10, 'r-o', label='NDCG@10', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Score')
axes[1].set_title('Evaluation Metrics')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150)
plt.show()
print("Saved: training_curves.png")

In [ ]:
# Visualize embeddings with t-SNE
model.eval()
with torch.no_grad():
    u_emb, i_emb = model.get_embeddings(train_edges.to(device))
    u_emb, i_emb = u_emb.cpu().numpy(), i_emb.cpu().numpy()

# Sample for speed
n = 300
u_sample = u_emb[np.random.choice(len(u_emb), n, replace=False)]
i_sample_idx = np.random.choice(len(i_emb), n, replace=False)
i_sample = i_emb[i_sample_idx]

combined = np.vstack([u_sample, i_sample])
print("Running t-SNE (may take a minute)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
coords = tsne.fit_transform(combined)

u_coords = coords[:n]
i_coords = coords[n:]

# Get movie genres for coloring
colors = []
genre_list = ['Action', 'Comedy', 'Drama', 'Sci-Fi', 'Romance']
for idx in i_sample_idx:
    mid = graph.idx_to_movie[idx]
    m = movies_df[movies_df['movie_id'] == mid].iloc[0]
    color = 'gray'
    for i, g in enumerate(genre_list):
        if m.get(g, 0) == 1:
            color = plt.cm.tab10(i)
            break
    colors.append(color)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].scatter(u_coords[:,0], u_coords[:,1], c='blue', alpha=0.5, s=20, label='Users')
axes[0].scatter(i_coords[:,0], i_coords[:,1], c='red', alpha=0.5, s=20, label='Movies')
axes[0].set_title('User vs Movie Embeddings')
axes[0].legend()

axes[1].scatter(i_coords[:,0], i_coords[:,1], c=colors, alpha=0.6, s=30)
axes[1].set_title('Movie Embeddings by Genre')
for i, g in enumerate(genre_list):
    axes[1].scatter([], [], c=plt.cm.tab10(i), label=g)
axes[1].legend()

plt.tight_layout()
plt.savefig('embeddings.png', dpi=150)
plt.show()
print("Saved: embeddings.png")

## Part 10: How Message Passing Captures Peer Influence

### The Magic of Multi-Hop Propagation:

**Layer 0 (Initial):**
- Each user/movie has a random embedding

**Layer 1 (1-hop):**
- User embedding ← aggregation of movies they liked
- Movie embedding ← aggregation of users who liked it

**Layer 2 (2-hop):**
```
User_A → Movie_X → User_B → Movie_Y
         (watched)  (also     (they
                    watched)   liked)
```

Now User_A's embedding contains info about:
- Movies they watched (1-hop)
- **Other users** who watched those movies (2-hop)  
- What those **similar users** liked (propagated through!)

### Why This Works:

1. **Similar users** (who liked similar movies) get **similar embeddings**
2. **Similar movies** (liked by similar users) get **similar embeddings**  
3. When we compute `dot(user_emb, movie_emb)`:
   - High score = embeddings align
   - The user embedding "points toward" movies they'd like!

### Collaborative Filtering Emerges Automatically!

No explicit "find similar users" step needed. The graph structure + message passing discovers these relationships naturally through the mathematics of aggregation.

---

### Congratulations! 🎉

You've built a working GNN-based recommendation system that:
1. Constructs a bipartite user-movie graph
2. Learns embeddings through message passing
3. Captures peer influence via multi-hop propagation
4. Generates personalized recommendations
5. Evaluates using Hit@K and NDCG@K